In [ ]:
import os
import glob
import rasterio
import numpy as np
from pathlib import Path
import utils

In [ ]:
archive = "/filserver/user/imagery"

combinations = [
    # ['Sentinel-1'],
    # ['Sentinel-2'],
    # ['Sentinel-1_Sentinel-2'],
    ['Sentinel-1', 'Sentinel-2']
    ]
years = [
    # 2020,
    2021,
    # 2022,
    # 2023
    ]
outputs = [
    # 's1',            # sar
    # 's2',            # msi
    # 's1s2_merge',    # msi w/sar bands
    's1s2_append'    # sar and msi
    ]


input_imgs = []
for combination in combinations:
    year_list = []
    for year in years:
        sensor_list = []
        for sensor in combination:
            sensor_list.extend(glob.glob(f"{archive}/{sensor}/{year}/*_segmscore.tiff"))
        year_list.append(sensor_list)
    input_imgs.append(year_list)
input_imgs

In [ ]:
for combination in combinations:
    for year in years:
        for sensor in combination:
            imgs = glob.glob(f"{archive}/{sensor}/{year}/*man2_segmscore.tiff")
            print(imgs)
            for img in imgs:
                os.remove(img)

In [ ]:
utils.show_single_band_img('/filserver/user/imagery/Sentinel-1_Sentinel-2/2021/S2A_MSIL2A_20210624T105031_N0500_R051_T32VNM_20230318T092732_aligned_sar_segmscore.tiff')

In [ ]:
def read_file(file):
    with rasterio.open(file) as src:
        return(src.read(4))

for i, comb in enumerate(combinations):
    post_fix = outputs[i]
    print(comb, post_fix)
    for year in years:
        print(year)
        input_imgs = []
        for sensor in comb:
            input_imgs.extend(glob.glob(f"{archive}/{sensor}/{year}/*_segmscore.tiff"))
        display(input_imgs)
        array_list = [read_file(x) for x in input_imgs]
        # display(array_list)
        array_mean = np.mean(array_list, axis=0)
        array_std = np.std(array_list, axis=0)

        # Get metadata from one of the input files
        with rasterio.open(input_imgs[0]) as src:
            meta = src.meta
        meta.update(count=2)
        meta.update(dtype=rasterio.float32)

        # Write output file
        output_file = os.path.join(archive, "results", str(year), f"mean_{post_fix}_{year}.tiff")
        with rasterio.open(output_file, mode='w', **meta) as dst:
            dst.write_band(1, array_mean.astype(rasterio.float32))
            dst.write_band(2, array_std.astype(rasterio.float32))
        print(output_file)

        utils.show_single_band_img(output_file, band=1)


In [ ]:
def read_file(file):
    print(file)
    with rasterio.open(file) as src:
        array_element = src.read()
        # display(array_element)
        return array_element

for i, comb in enumerate(combinations):
    post_fix = outputs[i]
    print(comb, post_fix)
    for year in years:
        print(year)
        input_imgs = []
        for sensor in comb:
            input_imgs.extend(glob.glob(f"{archive}/{sensor}/{year}/*_segmscore.tiff"))
        display(input_imgs)
        array_list = np.array([read_file(x) for x in input_imgs[:3]])
        # display(array_list.shape)
        array_mean_back = np.nanmean(array_list[:,0,...], axis=0)
        array_mean_built = np.nanmean(array_list[:,1,...], axis=0)
        array_mean_road = np.nanmean(array_list[:,2,...], axis=0)
        array_mean_inf = np.nanmean(array_list[:,3,...], axis=0)
        # array_std = np.std(array_list, axis=0)

        # Get metadata from one of the input files
        with rasterio.open(input_imgs[0]) as src:
            meta = src.meta
        meta.update(count=4)
        meta.update(dtype=rasterio.float32)

        # Write output file
        output_file = os.path.join(archive, "results", str(year), f"mean_{post_fix}_{year}_all.tiff")
        with rasterio.open(output_file, mode='w', **meta) as dst:
            dst.write_band(1, array_mean_back.astype(rasterio.float32))
            dst.write_band(2, array_mean_built.astype(rasterio.float32))
            dst.write_band(3, array_mean_road.astype(rasterio.float32))
            dst.write_band(4, array_mean_inf.astype(rasterio.float32))
        print(output_file)

        utils.show_single_band_img(output_file, band=1)

In [ ]:
for comb in outputs:
    mean_imgs = glob.glob(os.path.join(archive, 'results', f'*/mean_{comb}_*[!2022].tiff'), recursive=True)
    display(mean_imgs)

    meta = rasterio.open(mean_imgs[0]).meta
    meta.update(dtype=rasterio.float32)

    for i in range(len(mean_imgs) -1):
        before_mean = rasterio.open(mean_imgs[i]).read(1)
        after_mean = rasterio.open(mean_imgs[i+1]).read(1)
        difference_mean = after_mean - before_mean

        
        before_std = rasterio.open(mean_imgs[i]).read(2)
        after_std = rasterio.open(mean_imgs[i+1]).read(2)
        difference_std = after_std - before_std

        before_year = Path(mean_imgs[i]).stem.split("_")[-1]
        after_year = Path(mean_imgs[i+1]).stem.split("_")[-1]

        output_file = os.path.join(archive, 'results', f"difference_{comb}_{before_year}_{after_year}.tiff")
        with rasterio.open(output_file, mode='w', **meta) as dst:
            dst.write_band(1, difference_mean.astype(rasterio.float32))
            dst.write_band(2, difference_std.astype(rasterio.float32))
        # utils.show_single_band_img(output_file, band=1)
        print(output_file)

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt

geojson = "/filserver/user/imagery/results/ex_difference.geojson"
display(geojson)
gdf = gpd.read_file(geojson)
display(gdf)

In [ ]:
s2_2021 = '/filserver/user/imagery/results/2021/mean_s2_2021.tiff'
s2_2023 = '/filserver/user/imagery/results/2023/mean_s2_2023.tiff'
s2_dif = '/filserver/user/imagery/results/difference_s2_2021_2023_norm.tiff'

s1_2021 = '/filserver/user/imagery/results/2021/mean_s1_2021.tiff'
s1_2023 = '/filserver/user/imagery/results/2023/mean_s1_2023.tiff'
s1_dif = '/filserver/user/imagery/results/difference_s1_2021_2023_norm.tiff'

s1s2m_2021 = '/filserver/user/imagery/results/2021/mean_s1s2_merge_2021.tiff'
s1s2m_2023 = '/filserver/user/imagery/results/2023/mean_s1s2_merge_2023.tiff'
s1s2m_dif = '/filserver/user/imagery/results/difference_s1s2_merge_2021_2023_norm.tiff'

s1s2a_2021 = '/filserver/user/imagery/results/2021/mean_s1s2_append_2021.tiff'
s1s2a_2023 = '/filserver/user/imagery/results/2023/mean_s1s2_append_2023.tiff'
s1s2a_dif = '/filserver/user/imagery/results/difference_s1s2_append_2021_2023_norm.tiff'

s2_eo_2021 = "/filserver/user/imagery/Sentinel-2/2021/S2A_MSIL2A_20210624T105031_N0500_R051_T32VNM_20230318T092732_aligned.tiff"
s2_eo_2023 = "/filserver/user/imagery/Sentinel-2/2023/S2A_MSIL2A_20230624T104621_N0509_R051_T32VNM_20230624T170454_aligned.tiff"

In [ ]:
def normalize(band):
    band_min, band_max = (band.min(), band.max())
    return ((band-band_min)/((band_max - band_min)))

def chip(img, envelope_gdf):
    with rio.open(img) as src:
        # Crop the image based on the envelope around the polygon
        window = src.window(*envelope_gdf.total_bounds)
        img = src.read(1,window=window)
    return img

def chip_rgb(img, envelope_gdf):
    with rio.open(img) as src:
        # Crop the image based on the envelope around the polygon
        window = src.window(*envelope_gdf.total_bounds)
        r, g, b = (normalize(src.read(k,window=window)) for k in (4, 3, 2))
    return np.dstack((r, g, b))

In [ ]:
buffer_m = 200
geoms = gdf.geometry
bounds = gdf.total_bounds

ds = [bounds[2]-bounds[0], bounds[3]-bounds[1]]
min_d, max_d = min(ds), max(ds)
index_min = min(range(len(ds)), key=ds.__getitem__)
dif = max_d/2 - min_d/2
square_bound = bounds
square_bound[index_min::2] = [square_bound[index_min]-dif, square_bound[2+index_min]+dif]

buffer_box = list(map(sum, zip(square_bound, (-buffer_m, -buffer_m, buffer_m, buffer_m))))
bbox_geometry = gpd.GeoSeries([box(*buffer_box)])
envelope_gdf = gpd.GeoDataFrame(geometry=bbox_geometry)

fig, ax = plt.subplots(ncols=3, nrows=5, figsize=(6,10), constrained_layout=True)
[ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False) for ax in ax.ravel()]
ax[0,0].set_ylabel('EO')
ax[1,0].set_ylabel('SAR')
ax[2,0].set_ylabel('Merge')
ax[3,0].set_ylabel('Append')
ax[4,0].set_ylabel('Truth')
ax[4,0].set_xlabel('2021')
ax[4,1].set_xlabel('2023')
ax[4,2].set_xlabel('Difference')


img11 = chip(s2_2021, envelope_gdf)
img12 = chip(s2_2023, envelope_gdf)
img13 = chip(s2_dif, envelope_gdf)
# base1 = ax[0,0].imshow(img11, vmin=0,vmax=1)
# base2 = ax[0,1].imshow(img12, vmin=0,vmax=1)
# base3 = ax[0,2].imshow(img13, vmin=-1,vmax=1)
base1 = ax[0,0].imshow(img11, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
base2 = ax[0,1].imshow(img12, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
base3 = ax[0,2].imshow(img13, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=-1,vmax=1)
fig.colorbar(base1, ax=ax[0,0], location='top', shrink=0.7)
fig.colorbar(base2, ax=ax[0,1], location='top', shrink=0.7)
fig.colorbar(base3, ax=ax[0,2], location='top', shrink=0.7)


img21 = chip(s1_2021, envelope_gdf)
img22 = chip(s1_2023, envelope_gdf)
img23 = chip(s1_dif, envelope_gdf)
ax[1,0].imshow(img21, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[1,1].imshow(img22, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[1,2].imshow(img23, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=-1,vmax=1)

img31 = chip(s1s2m_2021, envelope_gdf)
img32 = chip(s1s2m_2023, envelope_gdf)
img33 = chip(s1s2m_dif, envelope_gdf)
ax[2,0].imshow(img31, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[2,1].imshow(img32, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[2,2].imshow(img33, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=-1,vmax=1)

img41 = chip(s1s2a_2021, envelope_gdf)
img42 = chip(s1s2a_2023, envelope_gdf)
img43 = chip(s1s2a_dif, envelope_gdf)
ax[3,0].imshow(img41, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[3,1].imshow(img42, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[3,2].imshow(img43, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=-1,vmax=1)

img51 = chip_rgb(s2_eo_2021, envelope_gdf)
img52 = chip_rgb(s2_eo_2023, envelope_gdf)
img53 = chip_rgb(s2_eo_2023, envelope_gdf)
ax[4,0].imshow(img51, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[4,1].imshow(img52, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=0,vmax=1)
ax[4,2].imshow(img53, extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]), vmin=-1,vmax=1)


geoms_s2 = gdf.loc[gdf.method=='eo'].geometry
for g in geoms_s2:
    x,y = g.exterior.xy
    ax[0,2].plot(x,y, color='red')
geoms_s1 = gdf.loc[gdf.method=='sar'].geometry
for g in geoms_s1:
    x,y = g.exterior.xy
    ax[1,2].plot(x,y, color='red')
geoms_m = gdf.loc[gdf.method=='merge'].geometry
for g in geoms_m:
    x,y = g.exterior.xy
    ax[2,2].plot(x,y, color='red')
geoms_a = gdf.loc[gdf.method=='append'].geometry
for g in geoms_a:
    x,y = g.exterior.xy
    ax[3,2].plot(x,y, color='red')
geoms_man = gdf.loc[gdf.method=='man'].geometry
for g in geoms_man:
    x,y = g.exterior.xy
    ax[4,2].plot(x,y, color='red')

plt.show()